In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import fire

In [2]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
user_data = pd.read_csv('../input/user.csv', encoding='latin-1')
book_data = pd.read_csv('../input/book.csv', encoding='latin-1')

C:\Users\z\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# book_data.fillna('', inplace=True)
# user_data.fillna('', inplace=True)

In [4]:
test_data.drop('id', axis=1, inplace=True)

In [5]:
train_m1 = pd.merge(train_data, user_data, on=["User-ID"], how='left')
train_m2 = pd.merge(train_m1, book_data, on=["ISBN"], how='left')

test_m1 = pd.merge(test_data, user_data, on=["User-ID"], how='left')
test_m2 = pd.merge(test_m1, book_data, on=["ISBN"], how='left')

In [6]:
train_m2.head()

,User-ID,ISBN,Book-Rating,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,8,0002005018,5,"timmins, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
1,8,0374157065,0,"timmins, ontario, canada",NaN,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
2,8,0393045218,0,"timmins, ontario, canada",NaN,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company
3,8,0399135782,0,"timmins, ontario, canada",NaN,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
4,8,0425176428,0,"timmins, ontario, canada",NaN,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group


In [7]:
test_m2.head()

,User-ID,ISBN,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,8,0060973129,"timmins, ontario, canada",NaN,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
1,8,0671870432,"timmins, ontario, canada",NaN,PLEADING GUILTY,Scott Turow,1993,Audioworks
2,8,074322678X,"timmins, ontario, canada",NaN,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner
3,8,0771074670,"timmins, ontario, canada",NaN,Nights Below Station Street,David Adams Richards,1988,Emblem Editions
4,14,0689821166,"mediapolis, iowa, usa",NaN,Flood : Mississippi 1927,Kathleen Duey,1998,Aladdin


In [8]:
train_m2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737864 entries, 0 to 737863
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              737864 non-null  int64  
 1   ISBN                 737864 non-null  object 
 2   Book-Rating          737864 non-null  int64  
 3   Location             737864 non-null  object 
 4   Age                  551760 non-null  float64
 5   Book-Title           737864 non-null  object 
 6   Book-Author          737863 non-null  object 
 7   Year-Of-Publication  737864 non-null  object 
 8   Publisher            737863 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 56.3+ MB


In [9]:
test_m2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206235 entries, 0 to 206234
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   User-ID              206235 non-null  int64  
 1   ISBN                 206235 non-null  object 
 2   Location             206235 non-null  object 
 3   Age                  150807 non-null  float64
 4   Book-Title           206235 non-null  object 
 5   Book-Author          206235 non-null  object 
 6   Year-Of-Publication  206235 non-null  object 
 7   Publisher            206234 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 14.2+ MB


In [10]:
train_m2['Age'].unique()

array([ nan,  37.,  42.,  62.,  34.,  28.,  27.,  38.,  15.,  41.,  16.,
        24.,  20.,  25.,  63.,  35.,  23.,  17.,  44.,  29.,  48.,  31.,
        40.,  51.,  26.,  21.,  14.,  36.,  22.,  33.,  50.,  19.,  49.,
        46.,  18.,  53.,   9.,  13.,  30.,  32.,  43.,  74.,  47.,  61.,
        60.,   3.,  55.,  45.,  66.,  59.,  71.,  57.,  52., 103.,  68.,
        12.,  65.,  77.,  39.,   2.,  69.,  70.,  56.,  54.,  11.,   8.,
       104.,  64., 239.,  58.,  73.,   1., 151., 201.,  67.,  72.,  76.,
         0., 128., 101., 141.,  78.,  75.,  96., 124., 209., 212.,  80.,
        81.,  10., 136.,  84.,  79.,   7.,  83.,  82., 108.,   4.,  92.,
        85.,  97., 168.,   5., 148.,  90., 152., 116., 229., 100.,  99.,
        93., 228., 107., 146., 204.,   6., 140.])

In [11]:
test_m2['Age'].unique()

array([ nan,  17.,  37.,  42.,  47.,  38.,  62.,  34.,  28.,  27.,  15.,
        20.,  41.,  16.,  24.,  25.,  63.,  45.,  35.,  23.,  44.,  29.,
        48.,  31.,  40.,  51.,  26.,  21.,  14.,  55.,  36.,  22.,  33.,
        43.,  50.,  52.,  19.,  49.,  46.,  18.,  53.,  32.,   9.,  13.,
        30.,  56.,  74.,  54.,  61.,  60.,   3.,  64.,  66.,  59.,  71.,
        57., 103.,  11.,  68.,  12.,  65.,  77.,  39.,  58.,   2.,  69.,
        70.,   8.,  73., 104.,  72., 239.,   1., 151., 201.,  67.,  76.,
         0., 123., 128., 101., 141.,  78.,  75.,  96., 124.,  81., 209.,
       212.,  80.,  10., 136.,  84.,  79.,  89.,   7.,  83.,  82., 108.,
         4.,  92.,  85., 147.,  97., 168.,   5., 148.,  90., 152., 116.,
       229., 100., 138.,  99.,  93., 228., 107., 146., 204.,  94.,   6.,
       200., 140.,  86.])

In [12]:
train_m2['Loc0'] = train_m2['Location'].apply(lambda x: x.split(',')[0].strip())
train_m2['Loc1'] = train_m2['Location'].apply(lambda x: x.split(',')[1].strip())
train_m2['Loc2'] = train_m2['Location'].apply(lambda x: x.split(',')[2].strip())

test_m2['Loc0'] = test_m2['Location'].apply(lambda x: x.split(',')[0].strip())
test_m2['Loc1'] = test_m2['Location'].apply(lambda x: x.split(',')[1].strip())
test_m2['Loc2'] = test_m2['Location'].apply(lambda x: x.split(',')[2].strip())

In [13]:
data = pd.concat([train_m2, test_m2], axis=0, ignore_index=True)

In [14]:
data['Book-Author'].fillna('', inplace=True)
data['Publisher'].fillna('', inplace=True)

In [15]:
def low_freq_encode(df, cat_cols, freq=3):
    for i in cat_cols:
        print(i)
        name_dict = df[i].value_counts().to_dict()
        df['{}_count'.format(i)] = df[i].apply(lambda x: 0 if name_dict[x] < freq else name_dict[x])
        df['{}_count'.format(i)] = df['{}_count'.format(i)].astype('category')
    return df

cat_cols = ['User-ID', 'Loc0', 'Loc1', 'Loc2', 'Book-Author', 'Publisher']
data = low_freq_encode(data, cat_cols)
data.drop(cat_cols, axis=1, inplace=True)

cat_cols = [i + '_count' for i in ['User-ID', 'Loc0', 'Loc1', 'Loc2', 'Book-Author', 'Publisher']]

data.drop(['Book-Title', 'ISBN', 'Location', 'Year-Of-Publication'], axis=1, inplace=True)

User-ID
Loc0
Loc1
Loc2
Book-Author
Publisher


In [16]:
train = data[data['Book-Rating'].notnull()]
test = data[data['Book-Rating'].isnull()]

cols = [i for i in train.columns if i not in ['Book-Rating']]
y = train['Book-Rating']
train = train[cols]
test = test[cols]

x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size=0.1, random_state=2200)

def train(iterations=22000, depth=10):
    model = CatBoostRegressor(
        iterations=iterations, 
        learning_rate=0.03,
        depth=depth,
        l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=2200,
        task_type="GPU"
    )
    model.fit(x_train,y_train, eval_set=(x_valid, y_valid), early_stopping_rounds=500, verbose=500, cat_features=cat_cols)

    result = model.predict(test_x)
    d = {"index":range(len(result)),"score":result}
    d = pd.DataFrame(d)
    d.to_csv("../sub/submission_{0}_{1}.csv".format(str(iterations),str(depth)), header=None, index=0)

train()

0:	learn: 2.6549465	test: 2.6387564	best: 2.6387564 (0)	total: 640ms	remaining: 3h 54m 44s
500:	learn: 2.3988135	test: 2.3524772	best: 2.3524772 (500)	total: 6m 32s	remaining: 4h 40m 56s
1000:	learn: 2.2514161	test: 2.1706430	best: 2.1706430 (1000)	total: 14m 9s	remaining: 4h 57m 1s
1500:	learn: 2.2001665	test: 2.1081019	best: 2.1081019 (1500)	total: 21m 43s	remaining: 4h 56m 47s
2000:	learn: 2.1780944	test: 2.0864016	best: 2.0864016 (2000)	total: 29m 1s	remaining: 4h 50m 4s
2500:	learn: 2.1643981	test: 2.0752722	best: 2.0752722 (2500)	total: 35m 34s	remaining: 4h 37m 21s
3000:	learn: 2.1547646	test: 2.0686306	best: 2.0686306 (3000)	total: 41m 54s	remaining: 4h 25m 21s
3500:	learn: 2.1464806	test: 2.0636672	best: 2.0636666 (3499)	total: 48m 8s	remaining: 4h 14m 21s
4000:	learn: 2.1400218	test: 2.0605410	best: 2.0605406 (3999)	total: 54m 14s	remaining: 4h 3m 58s
4500:	learn: 2.1349047	test: 2.0585626	best: 2.0585626 (4500)	total: 1h 10s	remaining: 3h 53m 55s
5000:	learn: 2.1307606	test:

NameError: name 'test_x' is not defined